In [25]:
import torch
import torch.nn as nn

Heads= no.of heads you are splitting

embed_size=256

### **1.Implementing Multihead attention**
#### 1.1 Class SelfAttention
- Define embed_size=256,  heads=8, self.head_dim=256//8==32, else raise assertion error
- Set values, keys, queries to embed_size

In [26]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        
        #for integer division
        self.head_dim = embed_size // heads
#self.head_dim=256/8=32, for eg:if we have 256/7, then that would not be possible, so we throw assertion error


#32*8==embed_size, i.e 256==256, if it does not raise an error
        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"

#defining linear layers 

#it just gonna map self.head_dim==32 to self.head_dim==32
        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)

        #after we concatenate
        #8*32(256)==(256)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

Y.shape is (n,m). So Y.shape[0] is n.

#### 1.2 forward function

- Initially, we give training examples
- Split the embedding i.e 256 into 8 heads
- Multiply queries with keys
- Mask
- Applying attention function
- returning output

In [27]:
def forward(self, values, keys, query, mask):
    
#first thing is we gonna give no.of training examples, how many examples we send at the same time
    N = query.shape[0]

#these lengths depends on where you use attention mechanism,is going to correspond source_len and target_len
#we are doing abstractly

        
    ## Split the embedding into self.heads different pieces, 256 into 8 heads
    #self.heads=8, self.head_dim=32
    #so previously the below was only one dimension i.e self.head_dim=32
    
    #lengths depends on where we use attn mechanism is used
        #corresponds to src len, target len, we just use abstractly
       
    value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]
    
    values = values.reshape(N, value_len, self.heads, self.head_dim)
    keys = keys.reshape(N, key_len, self.heads, self.head_dim)
    query = query.reshape(N, query_len, self.heads, self.head_dim)
 
  ##after reshaping we need to send them through linear layers
    
    values = self.values(values)  # (N, value_len, heads, head_dim)
    keys = self.keys(keys)  # (N, key_len, heads, head_dim)
    queries = self.queries(query)  # (N, query_len, heads, heads_dim)
    
    #multiply queries with the keys
        #output from energy
        #einsum -used for matrix multiplication      
    energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        # queries shape: (N, query_len, heads, heads_dim), :(n,q, 8, 32)
        # keys shape: (N, key_len, heads, heads_dim), :(n,k, 8, 32)
        # energy: (N, heads, query_len, key_len), :(n, 8,q, k)
        #"nqhd, nkhd--->nhqk"
        #q-query length,[queries, keys]
        #k-key length
        #h-heads
        #d-heads_dimension
        
        
        
        # Mask padded indices so their weights become 0
        #Mask for the target gonna be a triangular matrix
    if mask is not None:
        energy = energy.masked_fill(mask == 0, float("-1e20"))
           
        
        #run the attention(q,k,v)=softmax(qkt/sqrt(dk))V
        # Normalize energy values similarly to seq2seq + attention
        # so that they sum to 1. Also divide by scaling factor for
        # better stability
        #normalizing according key_len
    attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)
        # attention shape: (N, heads, query_len, key_len)
        
        
        
    out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )
        #l-dimension we are gonna multiply across
        # attention shape: (N, heads, query_len, key_len)
        # values shape: (N, value_len, heads, heads_dim)
        # output after (einsum) matrix multiply: (N, query_len, heads, head_dim), then
        # we reshape and flatten the last two dimensions.
        ##if we can see key_len, value_len are goin to be the same
        
    out = self.fc_out(out)
        # Linear layer doesn't modify the shape, final shape will be
        # (N, query_len, embed_size)

    return out
        
        
        
        

### **2. Implementing Transformers block i.e Multihead, feed-forward**

#### 2.1 Defining Transformers class

- We are going to send 256, 8, dropout, forward expansion
- 1.Multi-head --- attentionself.attention uses the attention we implemented above (256,8)
- 2.Normalization --- We are going to use 2 normalizations, sending through multihead, normalization, feedforward, normalization
- Defining Feedforward ---- forward_expansion (4 in the paper) * embed size, this block is just doing the extra computations

In [28]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )

        self.dropout = nn.Dropout(dropout)

#### 2.2 Defining forward function
- We add the skip connection using attention+query

In [29]:
def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)

        # Add skip connection, run through normalization and finally dropout
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out

### **3. Implementing encoder block**

#### 3.1 Class Encoder

- 1. Defining hyperparameters of a model, src_vocab_size, heads=8, num_layers== how many times you are repeating the encoder
- 2. Max_length is related to positional encoding. We need to be sending hwo long is the max sentence length
- 3. Word embedding - (src_vocab_size, embed_size)
- 4. Position Embedding - (max_length, embed_size)
- 5. Passes through layers (Transformers Block,....)


#### 3.2 Forward function
- 1. Calculating positions through positions formula
- 2. Combining word embedding and positional embedding

In [36]:
class Encoder(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        embed_size,
        num_layers,
        heads,
        device,
        forward_expansion,
        dropout,
        max_length,
    ):

        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)
        
        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion,
                )
                for _ in range(num_layers)
            ]
        )



        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        
        ##what we have from the begining
        N, seq_length = x.shape
        
        ##used how words are structured, permutations of the words
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        
        #word_embedding and position_embedding-- output
        out = self.dropout(
            (self.word_embedding(x) + self.position_embedding(positions))
        )

        # In the Encoder the query, key, value are all the same, it's in the
        # decoder this will change. This might look a bit odd in this case.
        #thats why we have out, out, out
        for layer in self.layers:
            out = layer(out, out, out, mask)

        return out

### **4. Implementing decoder block**

- Send it through masked multihead attention then use transformers block
- trg_mask=masked multi head attention

In [31]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()
        self.norm = nn.LayerNorm(embed_size)
        self.attention = SelfAttention(embed_size, heads=heads)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, trg_mask):
        attention = self.attention(x, x, x, trg_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(value, key, query, src_mask)
        return out

### **5. Implementing decoder**

In [32]:
class Decoder(nn.Module):
    def __init__(
        self,
        trg_vocab_size,
        embed_size,
        num_layers,
        heads,
        forward_expansion,
        dropout,
        device,
        max_length,
    ):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size, heads, forward_expansion, dropout, device)
                for _ in range(num_layers)
            ]
        )
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        x = self.dropout((self.word_embedding(x) + self.position_embedding(positions)))

        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)

        out = self.fc_out(x)

        return out

### **6. Implementing Transformers**

src_pad_idx,trg_pad_idx=used for masking

In [33]:
class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        trg_pad_idx,
        embed_size=512,
        num_layers=6,
        forward_expansion=4,
        heads=8,
        dropout=0,
        device="cpu",
        max_length=100,
    ):

        super(Transformer, self).__init__()

        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length,
        )

        self.decoder = Decoder(
            trg_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            device,
            max_length,
        )

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        # (N, 1, 1, src_len)
        return src_mask.to(self.device)

    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        #triangular matrix
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
            N, 1, trg_len, trg_len
        )

        return trg_mask.to(self.device)

    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        return out
        


In [35]:
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)

    x = torch.tensor([[1, 5, 6, 4, 3, 9, 5, 2, 0], [1, 8, 7, 3, 4, 5, 6, 7, 2]]).to(
        device
    )
    trg = torch.tensor([[1, 7, 4, 3, 5, 9, 2, 0], [1, 5, 6, 2, 4, 7, 6, 2]]).to(device)

    src_pad_idx = 0
    trg_pad_idx = 0
    src_vocab_size = 10
    trg_vocab_size = 10
    model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, device=device).to(
        device
    )

cuda


In [1]:
-nvidia smi

SyntaxError: invalid syntax (3905517398.py, line 1)